#### 1.1 Import Dependencies

In [4]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

#import tensorflow dependencies - functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

#### 1.2 Set GPU Growth

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#### 1.3 Folder Structure

In [6]:
REGISTERED_PATH = os.path.join('data', 'registered')
PROCESSED_PATH = os.path.join('data', 'processed')
os.makedirs(REGISTERED_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

#### 1.4 Data collection function

In [10]:
import uuid


'data\\registered\\e47da1ba-8fd3-11ef-8808-3c9c0f7c711b.jpg'

In [29]:
def collect_faces(user_id, name):
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Can't open camera.")
        return

    user_folder = os.path.join(REGISTERED_PATH, name)  
    os.makedirs(user_folder, exist_ok=True)  

    save_path = os.path.join(user_folder, f'user_{user_id}.jpg')  

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame.")
            break

        cv2.imshow('Camera', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            cv2.imwrite(save_path, frame) 
            print(f"Saved {save_path}")
            break
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    process_and_save_image(save_path) 

def process_and_save_image(img_path):
    img = cv2.imread(img_path)

    augmented_images = data_aug(img)

    user_folder = os.path.dirname(img_path) 

    for image in augmented_images:
        save_path = os.path.join(user_folder, '{}.jpg'.format(uuid.uuid1())) 
        cv2.imwrite(save_path, image.numpy()) 
user_id = uuid.uuid1() 
folder_name = "user_folder_name"
collect_faces(user_id, folder_name)


Saved data\registered\user_folder_name\user_142b8cf7-8fd7-11ef-bfc9-3c9c0f7c711b.jpg


## 3. Load and Preprocess Images

3.1 Get Image Directories

In [32]:
user_folder_path = os.path.join(REGISTERED_PATH, folder_name)

registered = tf.data.Dataset.list_files(os.path.join(user_folder_path, '*.jpg')).take(3000)

dir_test = registered.as_numpy_iterator()

first_image_path = dir_test.next().decode('utf-8')

data\registered\user_folder_name\user_142b8cf7-8fd7-11ef-bfc9-3c9c0f7c711b.jpg


3.2 Preprocessing - Scale and Resize

In [ ]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [ ]:
img = preprocess(first_image_path)